# **1. LIBRARIES AND SETUP**


In [ ]:
!pip install torch torchvision torchaudio
!pip install easyocr

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import easyocr
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os.path
from PIL import Image

# **2. Make directory + Importing Images**

In [ ]:
path = 'parsedimg'
os.mkdir(path)
img_path = './parsedimg'
img_path2 = 'full_schedule.png'

# **3. Split and sort the schedule into 5 images**

In [ ]:
# split all the images into 5
output_path = './parsedimg'
img = Image.open(img_path2)
x_width, y_height = img.size
outputFileFormat = "{0}-{1}.jpg"
baseName = "day"
pictures = 5
edges = np.linspace(0, x_width, pictures+1)
i = 0
for start, end in zip(edges[:-1], edges[1:]):
    box = (start, 0, end, y_height)
    a = img.crop(box)
    a.load()
    outputName = os.path.join(output_path, outputFileFormat.format(baseName, i + 1))
    a.save(outputName, "png")
    i += 1

In [ ]:
# Organize the images created in the folder into a sorted list
result = []
i = 0
"""if i == 0:
    i += 1
    continue
  else:"""
for images in os.listdir(img_path):
    images = './parsedimg/' + images
    result.append(images)
result = sorted(result)

# **4. Read and sort the days into courses**

In [ ]:
reader = easyocr.Reader(['en'], gpu=True)
days = []
for image in result:
  r = reader.readtext(image)
  e_text = []
  print(image)
  for detected_line in r:
    text = detected_line[1]
    e_text.append(text)
  days.append(e_text)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete./parsedimg/day-1.jpg
./parsedimg/day-2.jpg
./parsedimg/day-3.jpg
./parsedimg/day-4.jpg
./parsedimg/day-5.jpg


In [ ]:
# day 0 = monday -> day 4
courses = [] # (course info, index(days))
day_name = {0: "Monday",
            1: "Tuesday",
            2: "Wednesday",
            3: "Thursday",
            4: "Friday"} # This was for experimenting

schedule = {"Monday": [],
            "Tuesday": [],
            "Wednesday": [],
            "Thursday": [],
            "Friday": []}

for day in days:
  amount_of_courses = len(day) / 4 # because there are for lines per course
  # courses.append(([day[i:i + 4] for i in range(0, len(day), 4)]))
  courses.append(tuple(([day[i:i + 4] for i in range(0, len(day), 4)], day_name[days.index(day)])))

In [ ]:
# this cell is for testing
courses[2]

([['PHYS 10O4-LZB', '14602', '8.35 4m-11:25 am', 'TB 210'],
  ['PHYS 1004-A', '14582', '11:35am', '12:55 pm'],
  ['RB 2200', 'ECOR 1041-D', '12101', '1:05 pm-2:25 pm'],
  ['HS 1J01', 'NATH 1104-E', '13942', '2.35 pmn-3.55 pm'],
  ['AI 301', 'MATH 1104-ET', '13948', '4635'],
  ['pin-5:25 pin', 'TBA', 'GEOG 1020-B', '12918'],
  ['6.05 pIri-7:55 pIn', 'NN 231']],
 'Wednesday')

# **OCR TO JSON FUNCTION**